## Deep Q-Network

Схема обучения алгоритма DQN изображена на рисунке: 
<img src="files/dqn_rp_diagram.jpg">

Ключевыми отличиями от обычной аппроксимации нейронной сетью являются два новых элемента: replay buffer и target network.

#### 1. Replay buffer
Хранит информацию, которую получал агент во время взаимодействия со средой. Позволяет случайно сэмплировать эту информацию в виде кортежей: $<S_{t-1}, a_{t-1}, r_{t-1}, s_{t}, {done}>$.

#### 2. Target Network $Q_{\theta}^{target}$
Используется для вычисления loss функции Policy Net. 
$$L = { 1 \over N} \sum_i (Q_{\theta}(s,a) - [r(s,a) + \gamma \cdot max_{a'} Q_{\theta}^{target}(s', a')]) ^2.$$
Каждые $N$ шагов обновляется весами $Q_{\theta}$.

#### 3. Policy Net $Q_{\theta}$
Используется для взаимодействия с окружением. Обучается используя накопленный в Replay Buffer опыт.


In [5]:
# импортируем необходимые библиотеки

import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# если видеокарта доступна, то будем ее использовать
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

In [6]:
# определяем класс Q-Network

class QNetwork(nn.Module):
    def __init__(self):
        """
        определение сети
        """
        super().__init__()
        
        self.fc1 = nn.Linear(4, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        """
        определение графа вычислений
        :param x: вход
        :return: 
        """
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def sample_action(self, obs, epsilon):
        """
        сэмплирование действия
        :param obs: 
        :param epsilon: 
        :return: 
        """
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 1)
        else:
            return out.argmax().item()

### Задание 1. Реализуйтей методы класса ReplayBuffer

In [7]:
class ReplayBuffer():

    def __init__(self, max_size):
        """
        создаем структуру для хранения данных
        """
        #~~~~~~~~~~ Решение ~~~~~~~~~~~~~~~
        
         
        self.buffer = collections.deque(maxlen=max_size)
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        

    def put(self, transition):
        """
        помещаем данные в replay buffer
        :param transition: (s, a, r, next_s, done_mask)
        :return:
        """
        #~~~~~~~~~~ Решение ~~~~~~~~~~~~~~~
        
         
        self.buffer.append(transition)
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        

    def sample(self, n):
        """
        сэмплируем батч заданного размера
        :param n: размер мини-батча
        :return:
        """
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        # сэмплируем случайный батч и заполняем s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst
        #~~~~~~~~~~ Решение ~~~~~~~~~~~~~~~
        
         
        mini_batch = random.sample(self.buffer, n)

        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)

    def __len__(self):
        """
        возвращает размер replay buffer'а
        :return: len(replay_buffer)
        """
        #~~~~~~~~~~ Решение ~~~~~~~~~~~~~~~
        
         
        return len(self.buffer)
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        


### Задание 2. Заполните пропуски метода train

In [8]:
def train(q, q_target, replay_buffer, optimizer, batch_size, gamma, updates_number=10):
    """
    тренируем нашу архитектуру
    :param q: policy сеть
    :param q_target: target сеть
    :param replay_buffer:
    :param optimizer:
    :param batch_size: размер мини-батча
    :param gamma: дисконтирующий множитель
    :param updates_number: количество обновлений, которые необходимо выполнить
    :return:
    """
    for i in range(updates_number):
        # сэмплируем мини-батч из replay buffer'а
        s, a, r, s_prime, done_mask = replay_buffer.sample(batch_size)

        # получаем полезность, для выбранного действия q сети
        # q_a = 
        #~~~~~~~~~~ Решение ~~~~~~~~~~~~~~~
        
         
        q_out = q(s)
        q_a = q_out.gather(1, a)
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        

        # получаем значение max_q target сети и считаем значение target
        # target = 
        #~~~~~~~~~~ Решение ~~~~~~~~~~~~~~~
        
         
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        

        # определяем loss функцию, для q!
        loss = F.smooth_l1_loss(q_a, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [9]:
def run(learning_rate, gamma, buffer_max_size, batch_size, target_update_interval,
        replay_buffer_start_size, print_interval=20, n_episodes=2000):
    # создаем окружение
    env = gym.make('CartPole-v1')

    # создаем q и target_q
    q = QNetwork()
    q_target = QNetwork()

    # копируем веса q в target_q
    q_target.load_state_dict(q.state_dict())

    # создаем replay buffer
    replay_buffer = ReplayBuffer(max_size=buffer_max_size)

    score = 0.0

    # инициализируем оптимизатор, полученным lr
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(n_episodes):

        # постепенно изменяем eps с 8% до 1%
        epsilon = max(0.01, 0.08 - 0.01 * (n_epi / 200))

        s = env.reset()

        # выполянем 600 шагов в окружении и сохраняем, полученные данные
        for t in range(600):

            # получаем действие, используя сеть q
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)

            # выполняем действие в окружении
            s_prime, r, done, info = env.step(a)

            # добавляем данные в replay buffer
            done_mask = 0.0 if done else 1.0
            
            # сжимаем вознаграждения и добавляем в replay buffer
            replay_buffer.put((s, a, r / 100.0, s_prime, done_mask))

            s = s_prime

            score += r

            if done:
                break

        if len(replay_buffer) > replay_buffer_start_size:
            train(q, q_target, replay_buffer, optimizer, batch_size, gamma)

        if n_epi % target_update_interval == 0 and n_epi != 0:
            q_target.load_state_dict(q.state_dict())

        if n_epi % print_interval == 0 and n_epi != 0:
            print("# of episode :{}, avg score : {:.1f}, buffer size : {}, epsilon : {:.1f}%".format(
                n_epi, score / print_interval, len(replay_buffer), epsilon * 100))
            score = 0.0
    env.close()

### Задание 3. Проведите эксперименты с различными гиперпараметрами, для реализованной архитектуры

In [10]:
# определяем гиперпараметры
run(learning_rate=0.0005,
    gamma=0.98,
    buffer_max_size=50000,
    batch_size=32,
    target_update_interval=10,
    replay_buffer_start_size=2000)

### Задание 4. Уберите из модели Target Net и сравните, как это влияет на скорость обучения (постройте график сходимости)

## Дополнительные материалы:
https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html